In [38]:
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods

In [14]:
# retrieve data from api on the web
url = 'http://chargepoints.dft.gov.uk/api/retrieve/registry/format/csv'
chargepoint_df = pd.read_csv(url)

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_99147/372998491.py:3: DtypeWarning: Columns (5,10,11,16,17,18,54,56,57,58,59,60,61,62,63,64,65,66,67,68,69,79,90,101,112,115,119,122,123,126,130,133,134,137,141,144,148,152,155) have mixed types. Specify dtype option on import or set low_memory=False.
  chargepoint_df = pd.read_csv(url)


In [16]:
# get shape of data
chargepoint_df.head()

,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,connector8Type,connector8RatedOutputKW,connector8OutputCurrent,connector8RatedVoltage,connector8ChargeMethod,connector8ChargeMode,connector8TetheredCable,connector8Status,connector8Description,connector8Validated
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# save list of columns to text file
with open('chargepoint_columns.txt', 'w') as f:
    for item in chargepoint_df.columns:
        f.write("%s\n" % item)


In [26]:
# get electricity data with only sheet name "2.2.4" and skipping first 12 rows
electricity_df = pd.read_excel('data/table_224.xlsx', sheet_name='2.2.4', skiprows=12)
electricity_df.head()

,Year,Region [Note 1],PES area,Credit: Average variable unit price (£/kWh)[Note 2],Credit: Average fixed cost (£/year)[Note 3],Direct debit: Average variable unit price (£/kWh)[Note 2],Direct debit: Average fixed cost (£/year)[Note 3],Prepayment: Average variable unit price (£/kWh)[Note 2],Prepayment: Average fixed cost (£/year)[Note 3],Overall: Average variable unit price (£/kWh)[Note 2],Overall: Average fixed cost (£/year)[Note 3]
0,2010,Aberdeen,Northern Scotland,0.118970,61.636341,0.113210,49.321286,0.132477,23.799432,0.118306,48.789916
1,2010,Belfast,Northern Ireland,0.150255,0.000000,0.144601,0.000000,0.146475,0.000000,0.147227,0.000000
2,2010,Birmingham,West Midlands,0.117074,39.052210,0.110215,23.398073,0.116092,62.963998,0.113377,34.616904
3,2010,Canterbury,South East,0.113047,53.368463,0.107275,39.210238,0.114402,58.159460,0.109783,45.487996
4,2010,Cardiff,South Wales,0.122484,52.171721,0.117614,39.982620,0.135087,13.584988,0.122350,39.022561


In [28]:
# population data
population_df = pd.read_csv('data/ukdetailedtimeseries2001to2020/MYEB1_detailed_population_estimates_series_UK_(2020_geog21).csv')
population_df.head()

,ladcode21,laname21,country,sex,age,population_2001,population_2002,population_2003,population_2004,population_2005,...,population_2011,population_2012,population_2013,population_2014,population_2015,population_2016,population_2017,population_2018,population_2019,population_2020
0,E06000001,Hartlepool,E,1,0,525,502,516,553,531,...,601,595,576,551,497,576,507,535,515,517
1,E06000001,Hartlepool,E,1,1,544,526,498,526,550,...,573,615,601,567,553,512,570,515,532,519
2,E06000001,Hartlepool,E,1,2,548,548,551,507,521,...,580,586,621,600,576,562,534,576,515,528
3,E06000001,Hartlepool,E,1,3,556,546,548,553,508,...,600,578,584,625,598,579,565,530,583,520
4,E06000001,Hartlepool,E,1,4,606,549,553,540,554,...,600,616,593,568,620,604,591,576,533,580


In [32]:
# gdhi data
gdhi_df = pd.read_excel("data/regionalgrossdisposablehouseholdincomeallitlregions.xls", sheet_name="Table 3", skiprows=1)
gdhi_df.head()

,ITL level,ITL code,Region name,1997,1998,1999,2000,2001,2002,2003,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,UK,UK,United Kingdom,10757,10998,11498,12216,12631,12951,13277,...,16731,17376,18003,18503,19529,19686,20126,21019,21475,21440
1,Other,TLB,England,11016,11274,11795,12519,12935,13235,13553,...,17024,17706,18364,18888,19997,20171,20613,21550,22016,21962
2,ITL1,TLC,North East,9253,9321,9678,10284,10696,10896,11182,...,14040,14637,14935,15265,15872,15992,16332,16861,17266,17416
3,ITL2,TLC1,Tees Valley and Durham,9200,9170,9596,10176,10596,10755,10967,...,13772,14311,14573,15002,15541,15631,15984,16528,16900,17090
4,ITL3,TLC11,Hartlepool and Stockton-on-Tees,9264,9200,9693,10180,10571,10898,11144,...,14338,14766,14865,15318,15762,15877,16216,16692,17182,17485


In [59]:
path = "data/veh0105.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_fuel_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_fuel_df.columns = vehicle_fuel_df.iloc[3]
# drop first 4 rows
vehicle_fuel_df = vehicle_fuel_df.iloc[4:]
vehicle_fuel_df.head()

# renumber index
vehicle_fuel_df = vehicle_fuel_df.reset_index(drop=True)
vehicle_fuel_df.head()

3,Units,BodyType,Fuel [note 2],Keepership [note 3],ONS Sort [note 6],ONS Code [note 6],ONS Geography [note 6],2022 Q4,2022 Q3,2022 Q2,...,2012 Q1,2011 Q4,2011 Q3,2011 Q2,2011 Q1,2010 Q4,2010 Q3,2010 Q2,2010 Q1,2009 Q4
4,Thousands,Buses and coaches,Diesel,Company,1.0,K02000001,United Kingdom,110.929,112.035,113.141,...,[x],[x],[x],[x],[x],[x],[x],[x],[x],[x]
5,Thousands,Buses and coaches,Diesel,Company,2.0,K03000001,Great Britain,106.768,107.851,109.104,...,130.845,130.6,132.005,132.245,132.133,132.315,133.697,133.139,133.375,133.488
6,Thousands,Buses and coaches,Diesel,Company,3.0,E92000001,England,90.01,90.973,91.969,...,110.038,109.682,110.833,111.141,110.935,111.137,112.244,111.814,111.897,112.022
7,Thousands,Buses and coaches,Diesel,Company,4.0,E12000001,North East,4.103,4.209,4.204,...,5.667,5.678,5.571,5.289,5.253,5.341,5.42,5.377,5.376,5.351
8,Thousands,Buses and coaches,Diesel,Company,5.0,E06000047,County Durham,0.608,0.629,0.653,...,1.482,1.503,1.513,1.513,1.47,1.521,1.521,1.477,1.47,1.497


In [70]:
path = "data/veh0142.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_electricity_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_electricity_df.columns = vehicle_electricity_df.iloc[3]
# drop first 4 rows
vehicle_electricity_df = vehicle_electricity_df.iloc[4:]
vehicle_electricity_df.head()

# renumber index
vehicle_electricity_df = vehicle_electricity_df.reset_index(drop=True)
vehicle_electricity_df.head()

3,Units,BodyType,Fuel,Keepership [note 3],ONS Sort [note 6],ONS Code [note 6],ONS Geography [note 6],2022 Q3,2022 Q2,2022 Q1,...,2012 Q1,2011 Q4,2011 Q3,2011 Q2,2011 Q1,2010 Q4,2010 Q3,2010 Q2,2010 Q1,2009 Q4
0,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,1808.0,1707.0,1572.0,...,[x],[x],[x],[x],[x],[x],[x],[x],[x],[x]
1,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,1702.0,1602.0,1513.0,...,54.0,58.0,57.0,59.0,58.0,49.0,45.0,41.0,40.0,38.0
2,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,1343.0,1249.0,1194.0,...,51.0,55.0,54.0,57.0,55.0,46.0,43.0,39.0,38.0,36.0
3,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,21.0,12.0,12.0,...,7.0,12.0,12.0,13.0,14.0,7.0,5.0,[c],[c],[c]
4,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,[c],[c],[c],...,0.0,0.0,[c],[c],[c],[c],[c],0.0,0.0,0.0


In [71]:
# load a sheet with car models bought and quarter they were purchased
car_models_df = pd.read_csv('data/vehicle-licensing-statistics-2022-data-files/df_VEH0120_GB.csv')
car_models_df.head()

,BodyType,Make,GenModel,Model,Fuel,LicenceStatus,2022Q4,2022Q3,2022Q2,2022Q1,...,2003Q4,2002Q4,2001Q4,2000Q4,1999Q4,1998Q4,1997Q4,1996Q4,1995Q4,1994Q4
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,Licensed,17,17,18,18,...,0,0,0,0,0,0,0,0,0,0
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,Petrol,SORN,3,2,2,2,...,0,0,0,0,0,0,0,0,0,0
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,Licensed,24,27,26,26,...,0,0,0,0,0,0,0,0,0,0
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,Petrol,SORN,3,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Cars,ABARTH,ABARTH SPIDER,124 SPIDER MULTIAIR,Petrol,Licensed,982,996,1000,986,...,0,0,0,0,0,0,0,0,0,0


In [74]:
# get data with fuel prices
fuel_prices_df = pd.read_excel('data/Weekly_Fuel_Prices_120623.xlsx', sheet_name='All years', skiprows=7)
fuel_prices_df.head()

,Date,ULSP: Pump price (p/litre),ULSP: Diff on previous WEEK (p/litre),ULSP: Diff on previous YEAR (p/litre),Duty rate ULSP (p/litre),VAT (% rate) ULSP,ULSD: Pump price (p/litre),ULSD: Diff on previous WEEK (p/litre),ULSD: Diff on previous YEAR (p/litre),Duty rate ULSD (p/litre),VAT (% rate) ULSD
0,2003-06-09,74.590284,NaN,NaN,45.82,17.5,76.773394,NaN,NaN,45.82,17.5
1,2003-06-16,74.469143,-0.121141,NaN,45.82,17.5,76.689054,-0.084340,NaN,45.82,17.5
2,2003-06-23,74.423566,0.000000,NaN,45.82,17.5,76.620546,-0.068508,NaN,45.82,17.5
3,2003-06-30,74.352421,-0.071145,NaN,45.82,17.5,76.505260,-0.115286,NaN,45.82,17.5
4,2003-07-07,74.283737,-0.068684,NaN,45.82,17.5,76.457376,0.000000,NaN,45.82,17.5
